# Training of chatbot
### Date 28.06.24 to 29.06.24

In [1]:
import json

with open('intents.json') as file:
    data = json.load(file)
import nltk
from nltk.stem import WordNetLemmatizer


In [2]:
import numpy as np
import random
lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_words = ['?', '!', '.']
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
import pickle
for intent in data['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word in the sentence
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        # Add documents in the corpus
        documents.append((word_list, intent['tag']))
        # Add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# Sort classes
classes = sorted(list(set(classes)))

wordspikle=open('words.pkl','wb')
classespikle=open('classes.pkl','wb')
wrd=pickle.dump(words,wordspikle)
cls=pickle.dump(classes,classespikle)

# print(f"Classes: {classes}")
# print(f"Words: {words}")




In [4]:
training = []
output_empty = [0] * len(classes)
# Create the training set, bag of words for each sentence
for doc in documents:
    bag = []
    # List of tokenized words for the pattern
    word_patterns = doc[0]
    # Lemmatize each word - create base word, in attempt to represent related words
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # Create the bag of words array: 1 if word match found in current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    # Output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# Shuffle the features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)
# print(training)
# Create train and test lists. X - patterns, Y - intents



In [5]:
train_x = list(training[:, 0])
train_y = list(training[:, 1])
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer with number of neurons equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = tf.keras.optimizers.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fit the model
model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)
model.save('chatbot_model.h5')

Epoch 1/100


c:\Users\rahul\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\rahul\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0072 - loss: 2.5447      
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0564 - loss: 2.5091     
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0442 - loss: 2.4136     
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 360us/step - accuracy: 0.3187 - loss: 2.3472
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3164 - loss: 2.3219     
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3446 - loss: 2.2072 
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2742 - loss: 2.1579     
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4662 - loss: 2.0253 
Epoch 9/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2684 - loss: 2.1377 
Epoch 10/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5392 - loss: 1.9244 
Epoch 11/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4359 - loss: 1.8142 
Epoch 12/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.

In [17]:
# After training, save the tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Load the tokenizer for inference
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [20]:
# Load the tokenizer for inference
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

def chat_with_bot(input_text):
    sequences = tokenizer.texts_to_sequences([input_text])
    padded_sequences = pad_sequences(sequences, maxlen=72)  # Adjust according to your training
    prediction = model.predict(np.array(padded_sequences))
    response_index = np.argmax(prediction)
    response = index_to_response(response_index)
    return response

# Ensure 'index_to_response' correctly maps indices to responses
def index_to_response(index):
    if index in responses:
        return responses[index]
    else:
        return "Sorry, I don't understand that."
input_text = "Hello, how are you?"
response = chat_with_bot(input_text)
print(response)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Sorry, I don't understand that.
